# Câu1

In [6]:
import pandas as pd

# Tạo dữ liệu mẫu
df = pd.DataFrame({
    'A': [1, 2, 3, 4, 5],
    'B': [2, 3, 4, 5, 6]
})

# Lưu thành CSV
df.to_csv('data.csv', index=False)


In [10]:
import sqlite3

# Đọc dữ liệu từ CSV
df = pd.read_csv('data.csv')

# Tạo kết nối SQLite trong RAM
conn = sqlite3.connect(":memory:")
df.to_sql("my_table", conn, index=False, if_exists="replace")

# SQL phải tạo cột phụ để tính toán vì SQLite không hiểu trực tiếp biểu thức phức tạp
query = """
SELECT
    COUNT(*) AS n,
    SUM(A) AS sumA,
    SUM(B) AS sumB,
    SUM(A * B) AS sumAB,
    SUM(A * A) AS sumA2,
    SUM(B * B) AS sumB2
FROM my_table;
"""

# Thực thi truy vấn
stats = pd.read_sql_query(query, conn).iloc[0]

# Lấy các biến cần thiết
n = stats['n']
sumA = stats['sumA']
sumB = stats['sumB']
sumAB = stats['sumAB']
sumA2 = stats['sumA2']
sumB2 = stats['sumB2']

# Tính hệ số tương quan Pearson r
numerator = n * sumAB - sumA * sumB
denominator = ((n * sumA2 - sumA ** 2) * (n * sumB2 - sumB ** 2)) ** 0.5

correlation = numerator / denominator
print("Hệ số tương quan Pearson giữa A và B là:", correlation)


Hệ số tương quan Pearson giữa A và B là: 1.0


# Câu2

In [12]:
from scipy.stats import chi2_contingency

# Dữ liệu
data = {
    'Day': ['Day 1', 'Day 1', 'Day 1', 'Day 2', 'Day 2', 'Day 2', 'Day 3', 'Day 3', 'Day 3', 'Day 4', 'Day 4', 'Day 4'],
    'Car': ['A', 'B', 'C'] * 4,
    'Score': [8, 9, 7, 7.5, 8.5, 7, 6, 7, 8, 7, 6, 5]
}

df = pd.DataFrame(data)

# Phân nhóm điểm: thấp (≤6.5), trung bình (6.6–7.5), cao (>7.5)
df['ScoreGroup'] = pd.cut(df['Score'], bins=[0, 6.5, 7.5, 10], labels=['Low', 'Medium', 'High'])

# Tạo bảng tần số
contingency = pd.crosstab(df['Car'], df['ScoreGroup'])

# Kiểm định chi-squared
chi2, p, dof, expected = chi2_contingency(contingency)

print("Chi-squared =", chi2)
print("p-value =", p)
print("Degrees of freedom =", dof)
print("\nExpected frequencies:")
print(expected)

# Đánh giá kết quả
if p < 0.05:
    print("\n Có sự khác biệt đáng kể giữa các mẫu xe về phân bố điểm.")
else:
    print("\n Không có sự khác biệt đáng kể giữa các mẫu xe.")


Chi-squared = 0.8999999999999999
p-value = 0.9245608198515713
Degrees of freedom = 4

Expected frequencies:
[[1.         1.66666667 1.33333333]
 [1.         1.66666667 1.33333333]
 [1.         1.66666667 1.33333333]]

 Không có sự khác biệt đáng kể giữa các mẫu xe.


Kết quả thử nghiệm phụ thuộc vào cả mẫu xe và ngày. 
Tuy nhiên, ảnh hưởng của ngày có thể mạnh hơn, do điểm số thay đổi rõ rệt qua các ngày thử nghiệm.

Hãy chuyển đổi dữ liệu sang dạng quan hệ và thực hiện kiểm tra χ2

In [20]:
# Bước 1: Chuyển dữ liệu số sang dạng phân loại (dùng pd.cut)
df['A_cat'] = pd.cut(df['A'], bins=2, labels=['Thấp', 'Cao'])
df['B_cat'] = pd.cut(df['B'], bins=2, labels=['Thấp', 'Cao'])

# Bước 2: Tạo bảng quan hệ (contingency table)
contingency_table = pd.crosstab(df['A_cat'], df['B_cat'])

# Bước 3: Thực hiện kiểm định Chi-squared
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Kết quả
print("Bảng chéo:\n", contingency_table)
print("\nChi-squared =", chi2)
print("p-value =", p)
print("Degrees of freedom =", dof)
print("Expected frequencies:\n", expected)


Bảng chéo:
 B_cat  Thấp  Cao
A_cat           
Thấp      3    0
Cao       0    2

Chi-squared = 1.7013888888888888
p-value = 0.1921064408679386
Degrees of freedom = 1
Expected frequencies:
 [[1.8 1.2]
 [1.2 0.8]]


1. Giá trị p-value = 0.1921 > 0.05
-Không đủ bằng chứng thống kê để bác bỏ giả thuyết không H₀.
-Điều đó có nghĩa là không có mối liên hệ có ý nghĩa thống kê giữa hai biến phân loại A_cat và B_cat.
2. Chi-squared = 1.7014 là tương đối nhỏ
-Sự khác biệt giữa giá trị quan sát và giá trị kỳ vọng là không đáng kể.
3. Số bậc tự do = 1
-Phù hợp với bảng 2 hàng × 2 cột (2x2).

# Câu3

In [24]:
# 1. Kết nối đến cơ sở dữ liệu SQLite (hoặc tạo mới)
conn = sqlite3.connect(":memory:")  # dùng ":memory:" cho CSDL tạm thời
cursor = conn.cursor()

# 2. Tạo bảng flights và thêm dữ liệu mẫu
cursor.execute("""
CREATE TABLE flights (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    departure_time INTEGER
)
""")

# 3. Thêm dữ liệu mẫu
sample_data = [(830,), (1445,), (30,), (1230,), (None,)]
cursor.executemany("INSERT INTO flights (departure_time) VALUES (?)", sample_data)
conn.commit()

# 4. Truy vấn và chuyển đổi departure_time thành HH:MM bằng SQL
query = """
SELECT 
    departure_time,
    TIME(printf('%02d:%02d', departure_time / 100, departure_time % 100)) AS departure_time_converted
FROM flights
"""

# 5. Đọc kết quả vào DataFrame
df_result = pd.read_sql_query(query, conn)
print(df_result)

# Đóng kết nối
conn.close()


   departure_time departure_time_converted
0           830.0                 08:30:00
1          1445.0                 14:45:00
2            30.0                 00:30:00
3          1230.0                 12:30:00
4             NaN                 00:00:00


1. Các giá trị departure_time dạng số nguyên đã được chuyển thành định dạng thời gian HH:MM:SS.

2. Thay vì hiển thị số 830, 1445, 30 vốn khó hiểu, giờ đây chúng ta thấy rõ 08:30:00, 14:45:00, 00:30:00.

# Câu4

In [32]:
import numpy as np

# Kết nối SQLite
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Tạo bảng mẫu và thêm dữ liệu
cursor.execute("CREATE TABLE data (id INTEGER, value REAL)")
values = [(1, 10), (2, 12), (3, 14), (4, 100), (5, 13), (6, 11)]
cursor.executemany("INSERT INTO data (id, value) VALUES (?, ?)", values)
conn.commit()

# Đọc dữ liệu vào pandas để tính MAD
df = pd.read_sql("SELECT * FROM data", conn)

# Tính median và MAD
median = df['value'].median()
mad = np.median(np.abs(df['value'] - median))

# Ngưỡng phát hiện ngoại lệ
threshold = 1.5 * mad

# Gắn cờ ngoại lệ
df['is_outlier'] = np.abs(df['value'] - median) > threshold

# Hiển thị kết quả
print(df[df['is_outlier'] == True])


   id  value  is_outlier
0   1   10.0        True
3   4  100.0        True


1. Giá trị 10.0 cũng được xem là ngoại lệ:

Vì nó nằm xa về phía dưới so với median, vượt quá ngưỡng MAD.
Điều này cho thấy ngoại lệ không chỉ là giá trị lớn, mà còn là những giá trị cực nhỏ nằm ngoài "vùng trung tâm" của dữ liệu.

2. Giá trị 100.0 là ngoại lệ rõ ràng:

Đây là một giá trị rất lớn so với phần còn lại của tập dữ liệu (khoảng 10–14), nên việc gắn cờ là hợp lý.

# Câu5